In [2]:
import numpy as np
import pandas as pd
import json

## 血常规报告参数

In [57]:
xcgbaogao = pd.read_csv(r"E:\Dataset\heretolymph\20240703新增\disease_arms\血常规通道参数.csv", encoding='gbk')
xcgbaogao.to_csv(r"E:\Dataset\heretolymph\20240703新增\disease_arms\血常规通道参数.csv", index=False, encoding='utf-8')

In [58]:
# 镜检这一列 均值 填充
xcgbaogao['住院号'] = xcgbaogao['住院号'].fillna(0)

In [59]:
# 0 填充其他列
xcgbaogao = xcgbaogao.fillna(xcgbaogao.mean())
xcgbaogao = xcgbaogao.fillna(0)


In [55]:
# 序号这一列 + 1000
xcgbaogao['序号'] = xcgbaogao['序号'] + 1000

In [60]:
# 保存，小数点后两位
xcgbaogao.to_csv(r"E:\Dataset\heretolymph\20240703新增\disease_arms\血常规通道参数1.csv", index=False, float_format='%.2f')

In [3]:
def fill_mean(csv_path, save_path):
    data = pd.read_csv(csv_path)
    data['住院号'] = data['住院号'].fillna(0)
    data['序号'] = data['序号'] + 1000
    data = data.fillna(-1)
    data.to_csv(save_path, index=False, float_format='%.1f')

file_src = r"E:\Dataset\heretolymph\20240626新增\验证数据 24.6.17\csv_data\control_arms\血常规通道参数0.csv"
file_des = r"E:\Dataset\heretolymph\20240626新增\验证数据 24.6.17\csv_data\control_arms\血常规通道参数1.csv"
fill_mean(file_src, file_des)

## 图片重新命名

In [20]:
# 将 ../ALL文件夹下的图片名称前加上 1000
import os
import shutil

def rename_files(path):
    files = os.listdir(path)
    for file in files:
        if file.endswith(".PNG"):
            # new_name = "2" + file[1:]
            new_name = file.lower()
            os.rename(os.path.join(path, file), os.path.join(path, new_name))

rename_files(r"E:\Dataset\heretolymph\20240626新增\验证数据 24.6.17\反应性疾病组 散点图")

## 转换为JSON格式

In [66]:
import os
arms = "control_arms"
os.chdir(r"E:\Dataset\heretolymph\20240703新增\\" + arms)

def transform(xcg_baogao, xcg_leibie, xcg_tongdao):
    xcg_baogao = pd.read_csv(xcg_baogao)
    xcg_leibie = pd.read_csv(xcg_leibie)
    xcg_tongdao = pd.read_csv(xcg_tongdao)
    for i in xcg_baogao.index:
        assert xcg_baogao.loc[i, "序号"] == xcg_leibie.loc[i, "序号"] == xcg_tongdao.loc[i, "序号"], "各项系数中的病人编号需要相同"
            
        p_no = xcg_baogao.loc[i, "序号"]
        # init
        data = {"基础信息":{}, "图片路径":"","临床":{}, "生化":{}, "血常规报告参数":{}, "淋巴类别":{}, "血常规通道参数":{}, "镜检":None}

        data["基础信息"] = xcg_baogao.iloc[i, 0:3].to_dict()
        data["图片路径"] = f"../{arms}/{p_no}.png"
        data["临床"] = None # linchuang.iloc[i, 3:].to_dict()
        data["生化"] = None # shenghua.iloc[i, 3:].to_dict()
        # data["生化"] = None
        data["血常规报告参数"] = xcg_baogao.iloc[i, 3:-1].to_dict()
        data["淋巴类别"] = xcg_leibie.iloc[i, 3:-1].to_dict()
        data["血常规通道参数"] = xcg_tongdao.iloc[i, 3:-1].to_dict()
        data["镜检"] = int(xcg_tongdao.iloc[i, -1])
        data['label'] = int(str(p_no)[0]) - 1


        des_path = f"../{p_no}.json"
        
        with open(des_path, 'w') as f:
            json.dump(data, f, indent=4, ensure_ascii=False)

transform("血常规报告参数1.csv", "血常规淋巴类别1.csv", "血常规通道参数1.csv")

In [67]:
os.chdir(r"E:\Dataset\heretolymph\20240703新增")
data = []
for file in os.listdir():
    if file.endswith("json"):
        with open(file, 'r') as f:
            data.append(json.load(f))

with open("__all_external.json", 'w') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

In [8]:
data = json.loads(open("__all.json", 'r').read())